<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphicBASE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Условия:**
1. наличие AUTODESK DESKTOP
2. доступ ко всем таблицам в режиме РЕДАКТОР
3. все тактические графики должны быть приведены в гугл форму

# Библиотеки

In [1]:
!pip install pygsheets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 KB 4.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.2 requires google-auth-oauthlib<0.5,>=0.4.1, but you have google-auth-oauthlib 1.0.0 which is incompatible.


In [2]:
import pygsheets

In [3]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import pandas as pd
import gspread_dataframe as gd

In [4]:
idx = pd.IndexSlice

In [5]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

# отчетные даты

In [6]:
# старт-финиш проекта
start_project = '01.02.2022'
start_project = pd.to_datetime(start_project)
finish_project = '02.09.2024'
finish_project = pd.to_datetime(finish_project)

In [7]:
# страт-финиш отчетной 
start_week = '11.02.2023'
start_week = pd.to_datetime(start_week)
finish_week = '20.02.2023'
finish_week = pd.to_datetime(finish_week)

In [8]:
# старт-финиш next недели, за отчетной
start_next_week = '21.02.2023'
start_next_week = pd.to_datetime(start_next_week)
finish_next_week = '28.02.2023'
finish__next_week = pd.to_datetime(finish_next_week)

# Предобработка GS: график проектирования, график СМР, график заказчика

## Реестр выдачи РД

https://docs.google.com/spreadsheets/d/1ZuZv1Rx-mp3AS5LFPjbEa-P92Xux_o_rvaOQb6NWXkE/edit#gid=479522789

In [9]:
# форма для обновления графика заказчика
worksheet_PD = gc.open('Реестр выдачи РД')
graph_PD = worksheet_PD.worksheet('График ПИР ')
graph_PD = graph_PD.get_all_values()
graph_PD = np.array(graph_PD)
graph_PD = pd.DataFrame(graph_PD[7:])
graph_PD

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,Корректировка Проектной документации,,,09.12.2022,,,,,,,,,,
1,1,,,Общая пояснительная записка,"ООО ""ИБС""",24.06.2022,08.12.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,
2,2,,,Схема планировочной организации земельного уча...,"ООО ""ИБС""",24.06.2022,21.11.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,
3,3,,,Архитектурные решения,"ООО ""ИБС""",24.06.2022,09.12.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,
4,4,,,Конструктивные решения ГЧ,"ООО ""ИБС""",24.06.2022,25.11.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,227,,,Первчиное консультирование по СМР проведение т...,"ООО ""Экостандарт""",06.10.2022,20.11.2022,STATUS A,FALSE,TRUE,FALSE,1,1,0,0,0,
244,228,,,Сопровождение строительства,"ООО ""Экостандарт""",06.10.2022,06.05.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,434,427,
245,229,,,Отчет по вводу объекта в эксплуатацию,"ООО ""Экостандарт""",06.05.2024,03.06.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,462,455,
246,230,,,Энергетическое моделирование,"ООО ""Экостандарт""",20.06.2023,31.10.2023,HOLD,TRUE,FALSE,FALSE,0,1,0,246,239,


In [10]:
# форма для обновления графика подрядчика
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())
graph_PD_RD

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,73,Просрочено,,START,FINISH,,,,,,,,,,
1,,,2,До конца срока менее 1 недели,,,,,,,,,,,,,
2,,,12,До конца срока менее 2-х недель,,,,,,,,,,,,,
3,,,81,Выполнено,,,,,,,,,,,,,
4,,20.02.2023,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,227,,,Первчиное консультирование по СМР проведение т...,"ООО ""Экостандарт""",06.10.2022,20.11.2022,STATUS A,FALSE,TRUE,FALSE,1,1,0,0,0,
251,228,,,Сопровождение строительства,"ООО ""Экостандарт""",06.10.2022,06.05.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,434,427,
252,229,,,Отчет по вводу объекта в эксплуатацию,"ООО ""Экостандарт""",06.05.2024,03.06.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,462,455,
253,230,,,Энергетическое моделирование,"ООО ""Экостандарт""",20.06.2023,31.10.2023,HOLD,TRUE,FALSE,FALSE,0,1,0,246,239,


## План работ Договор Адамант

https://docs.google.com/spreadsheets/d/1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc/edit#gid=1411311560

In [ ]:
# форма для обновления графика заказчика
worksheet_SMR = gc.open('План работ Договор Адамант')
graf = worksheet_SMR.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

In [12]:
# форма для заполнения факта
graf_black = graf.copy()

In [13]:
# форма для обновления графика недели
week_graf = graf.copy()
week_graf = week_graf.loc[idx[2:],idx[:6]]
week_graf.drop([4,5], axis=1, inplace=True)
week_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
week_graf = week_graf[week_graf['passwordCOST'] > '']
week_graf = week_graf[week_graf['START'] > '']
week_graf = week_graf[week_graf['FINISH'] > '']
week_graf = week_graf.set_index('passwordCOST')
week_graf['valueModel'] = week_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
week_graf = week_graf.fillna(0)
week_graf

,NAME,START,FINISH,valueModel
passwordCOST,,,,
VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,0.0
VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,300.0
VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,2.0
VZO_POS1_POD_VRS_VK0.01,Временный водопровод и канализация (с подогрев...,19.10.2022,14.11.2022,1.0
VZO_POS1_POD_VRD_DP17.02,Устройство основания под бытовые помещения (по...,16.10.2022,09.11.2022,200.0
...,...,...,...,...
BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,0.0
BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,0.0
_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,0.0


In [14]:
# общий график работ
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])
graf_total = graf.drop(graf.index[1:10], axis=1)
newColumnsIndex = [pd.to_datetime(pd.Series(graf_total.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf_total.iloc[1][1:].values]
graf_total.rename(columns={0: 'passwordCOST'}, inplace=True)
graf_total = graf_total.set_index("passwordCOST")
graf_total = pd.DataFrame(graf_total.values, index=graf_total.index, columns=newColumnsIndex)
graf_total.drop(graf_total.index[:2], axis=0, inplace=True)
for i in graf_total.columns:
    graf_total[i] = graf_total[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))
graf_total.fillna(0, inplace=True)
graf_total.sort_index(axis=0, level=1, ascending=True)
graf_total = graf_total[graf_total.index > '']
graf_total

2022-10-10      2022-10-11      2022-10-12       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2022-10-13      2022-10-14       ... 2023-02-23       \
                               план факт       план факт  ...       план факт   
passwordCOST                                              ...                   
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0  ...        0.0  0.0   
...                             ...  ...        ...  ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0  ...        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0  ...        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   

                         2023-02-24      2023-02-25      2023-02-26       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2023-02-27       
                               план факт  
passwordCOST                              
VZO_POS1_POD_MOB_.              0.0  0.0  
VZO_POS1_POD_VRD_DP17.01        0.0  0.0  
VZO_POS1_POD_VRS_ES0.01         0.0  0.0  
VZO_POS1_POD_VRS_VK0.01         0.0  0.0  
VZO_POS1_POD_VRD_DP17.02        0.0  0.0  
...                             ...  ...  
BOT_GP___.                      0.0  0.0  
BOT_GP                          0.0  0.0  
_EXP___.                        0.0  0.0  
_ZOS___.                        0.0  0.0  
_RNV___.                        0.0  0.0  

[374 rows x 282 columns]

In [15]:
# перечень работ -> вгрузка из Plan Radar
plan_PR = worksheet_SMR.worksheet('PR')
plan_PR = plan_PR.get_all_values()
plan_PR = np.array(plan_PR)
plan_PR = pd.DataFrame(plan_PR)
plan_PR = plan_PR.iloc[1:, [0,7,9]]
plan_PR = pd.DataFrame(plan_PR)
plan_PR

,0,7,9
1,VZO_POS1_POD_INF_STN.01,1.0,2022-10-27
2,OOS_KR1_ZEM_VP_00.01,1.0,2022-11-02
3,VZO_POS1_POD_VRD_DP17.01,140.0,2022-11-02
4,VZO_POS1_POD_VRD_DP17.02,80.0,2022-11-02
5,VZO_POS1_POD_VRS_VK0.01,1.0,2022-11-02
...,...,...,...
129,OOS_KZ01_IZO_GFP_CP150.02,339.8,2023-02-15
130,OOS_KZ01_IZO_GFP_CP150.03,276.94,2023-02-15
131,OOS_KZ01_MEX_BFP_AAK.01,10.0,2023-02-07
132,OOS_KR3_IZO_GFP_SPO.01,61.56,2023-02-15


## График по форме заказчика

In [16]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

Saving Сводный план-график 2023 01 27.xlsx to Сводный план-график 2023 01 27.xlsx


In [17]:
df_graph = pd.read_excel(open('Сводный план-график 2023 01 27.xlsx', 'rb'))
df_graph

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Рекомендации по фильтру:,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,Статус задач на 31.12.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. В фильтре по столбцу ""Названия ресурсов"" на...",NaN,NaN,NaN,1. Уровень. Группа
1,NaN,NaN,КТ1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Уровень. Суммарная задача
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Внесено изменение,СДР,Название задачи,Нарушен срок,%,План %,Откл. План%,Столбец2,Столбец1,Начало,Окончание,Отв. Исполнитель,Названия ресурсов,Столбец3,Столбец4
4,NaN,0,Проект Школа Новый Вгляд 2022 12 27,0,0.21,24,3,Вт 18.01.22,Ср 10.12.25,2022-01-18 00:00:00,2025-12-10 00:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,NaN,16.3.10,ПО бухгалтерского и финансово-экономи...,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1251,NaN,16.3.11,ПО зарплаты и управления персоналом,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1252,NaN,16.3.12,ПО автоматизации медицинского пункта,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1253,NaN,16.3.13,ПО системы заказа пропусков,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN


## Реестр замечаний из ACC

In [18]:
# файл с реестром замечаний на google drive
from google.colab import files
upload = files.upload()

Saving Сводка по проблемам-202302200523.xlsx to Сводка по проблемам-202302200523.xlsx


In [19]:
value_essuse = pd.read_excel(open('Сводка по проблемам-202302200523.xlsx', 'rb'), sheet_name=1)
value_essuse = value_essuse[(value_essuse['Статус'] == 'Открыто') & (value_essuse['Тип'].isin(['Замечание к РД','Замечания к ПД']))]
value_essuse.loc[:, 'Размещение'] =  value_essuse['Размещение'].map(lambda x: str(x).split('.pdf')[0].split('_')[0])
value_essuse = value_essuse.loc[:, idx['Размещение','Тип замечания']]
agg_func_count = {'Тип замечания': 'count'}
value_essuse = value_essuse.groupby(['Размещение','Тип замечания']).agg(agg_func_count)
value_essuse

Тип замечания
Размещение              Тип замечания                       
608AR-20-П-ИОС5.4.1-ОВ1 Техническая ошибка                 1
608AR-20-Р-ВК-2         Критическое замечание              1
608AR-20-Р-ВК-3         Критическое замечание              1
608AR-20-Р-КЖ-10-1      Критическое замечание              1
                        Недостаточно данных                1
                        Оформление                         1
                        Техническая ошибка                 3
608AR-20-Р-КЖ-10-2      Критическое замечание              2
                        Недостаточно данных                1
                        Техническая ошибка                 1
608AR-20-Р-КЖ-10-3      Критическое замечание              1
                        Техническая ошибка                 1
608AR-20-Р-КЖ-6-2       Техническая ошибка                 1
608AR-20-Р-ОЗДС         Недостаточно данных                3
                        Оформление                         1
608AR-20-Р-СОТС         Недостаточно данных                1
608AR-20-Р-ЭО           Критическое замечание              4
                        Оформление                         2
                        Техническая ошибка                 6

# **Обновление графика проектирования подрядчика: 'График ПИР '**

* выгрузка реестра замечаний 
* агрегирование данных по разделам
* заполнение графика проектирования

In [20]:
name_RD_issues = pd.Series(value_essuse.index.get_level_values(0)).unique()

In [21]:
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())

In [22]:
list_PD_RD = graph_PD_RD.iloc[:,2].to_list()

In [23]:
for i in name_RD_issues:
    try:
        number_row = list_PD_RD.index(i) + 1
        try:
            value_essuse.loc[i, 'Критическое замечание']
            worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS C')
        except: 
            try:
                value_essuse.loc[i, 'Техническая ошибка'].values[0] != 0  
                value_essuse.loc[i, 'Недостаточно данных'].values[0] != 0
                worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS B')
            except:
                try:
                    worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS A') 
                except:
                    pass
    except:
        pass 

# **Обновление графика СМР подрядчика: 'PLAN_WEEK', 'START', 'ГРАФИК РАБОТ'**

* в модели отмечается выполненный элемент
* данные попадают в GS + обновляется формы PLAN RADAR

## срез на отчетную неделю, план на следующую неделю за отчетной, кумулятивная сумма

In [24]:
graf_week_plan = graf_total.loc[:, idx[start_week:finish_week,'план']].sum(axis=1).values
graf_week_plan = pd.DataFrame(graf_week_plan,index=graf_total.index, columns=['PLAN_WEEK'])
graf_week_fakt = graf_total.loc[:, idx[start_week:finish_week,'факт']].sum(axis=1).values
graf_week_fakt = pd.DataFrame(graf_week_fakt,index=graf_total.index, columns=['FAKT_WEEK'])

In [25]:
graf_next_week_plan = graf_total.loc[:, idx[start_next_week:finish_next_week,'план']].sum(axis=1).values
graf_next_week_plan = pd.DataFrame(graf_next_week_plan,index=graf_total.index, columns=['PLAN_NEXT_WEEK'])
graf_next_week_fakt = graf_total.loc[:, idx[start_next_week:finish_next_week,'факт']].sum(axis=1).values
graf_next_week_fakt = pd.DataFrame(graf_next_week_fakt,index=graf_total.index, columns=['FAKT_NEXT_WEEK'])

In [26]:
total_plan = graf_total.loc[:, idx[start_project:finish_week,'план']].sum(axis=1).values
total_plan = pd.DataFrame(total_plan,index=graf_total.index, columns=['TOTAL_PLAN'])
total_fakt = graf_total.loc[:, idx[start_project:finish_week,'факт']].sum(axis=1).values
total_fakt = pd.DataFrame(total_fakt,index=graf_total.index, columns=['TOTAL_FAKT'])

In [27]:
new_graf_week = week_graf.join([total_plan,total_fakt,graf_week_plan,graf_week_fakt,graf_next_week_plan,graf_next_week_fakt])

In [28]:
new_graf_week = new_graf_week[['NAME', 'TOTAL_PLAN','TOTAL_FAKT','PLAN_WEEK','FAKT_WEEK','PLAN_NEXT_WEEK','FAKT_NEXT_WEEK']]

In [29]:
new_graf_week = new_graf_week.loc[(new_graf_week['PLAN_WEEK']>0) | (new_graf_week['PLAN_NEXT_WEEK']>0)]

In [30]:
new_graf_week['%TOTAl_WEEK'] =( new_graf_week['FAKT_WEEK'] / new_graf_week['PLAN_WEEK']).mul(100).round(2).astype(str).add(' %')

In [31]:
new_graf_week['%TOTAL'] =( new_graf_week['TOTAL_FAKT'] / new_graf_week['TOTAL_PLAN']).mul(100).round(2).astype(str).add(' %')

In [32]:
new_graf_week.rename(columns={'NAME':f'WORK on {pd.to_datetime(start_week).date()}  to {pd.to_datetime(finish_week).date()}'}, inplace=True)

In [33]:
worksheet_SMR.worksheet('PLAN_WEEK').clear()
worksheet_SMR.worksheet('PLAN_WEEK').append_rows([new_graf_week.columns.values.tolist()]+new_graf_week.values.tolist())

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'updates': {'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
  'updatedRange': 'PLAN_WEEK!A1:I15',
  'updatedRows': 15,
  'updatedColumns': 9,
  'updatedCells': 135}}

In [34]:
# справочно
dir(worksheet_SMR.worksheet('PLAN_WEEK'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_finder',
 '_properties',
 'acell',
 'add_cols',
 'add_rows',
 'append_row',
 'append_rows',
 'batch_get',
 'batch_update',
 'cell',
 'clear',
 'clear_basic_filter',
 'client',
 'col_count',
 'col_values',
 'delete_row',
 'delete_rows',
 'duplicate',
 'export',
 'find',
 'findall',
 'format',
 'freeze',
 'frozen_col_count',
 'frozen_row_count',
 'get',
 'get_all_records',
 'get_all_values',
 'id',
 'insert_row',
 'merge_cells',
 'range',
 'resize',
 'row_count',
 'row_values',
 'set_basic_filter',
 'sort',
 'spreadsheet',
 'title',
 'update',
 'update_acell',
 'update_cell',
 'update_cells',
 'update_index',
 'update_tit

## данные по стартовым работам -> триггер для Plan Radar

In [35]:
start = start_project
finish = pd.to_datetime(pd.Timestamp.today().date())
start, finish

(Timestamp('2022-01-02 00:00:00'), Timestamp('2023-02-20 00:00:00'))

In [36]:
finish_yesterday = finish - pd.Timedelta(1, "d")

In [37]:
total_plan = graf_total.loc[:, idx[start:finish,'план']].sum(axis=1).values
cum_sum_yesterday = graf_total.loc[:, idx[start:finish_yesterday,'план']].sum(axis=1).values
sum_today = graf_total.loc[:, idx[finish:finish,'план']].sum(axis=1).values
# total_plan_value = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values - graf.loc[:, idx[start:finish,'план']].sum(axis=1).values

In [38]:
total_plan = pd.DataFrame(total_plan,index=graf_total.index, columns=['cum_sum_today'])
cum_sum_yesterday = pd.DataFrame(cum_sum_yesterday,index=graf_total.index, columns=['cum_sum_yesterday'])
sum_today = pd.DataFrame(sum_today,index=graf_total.index, columns=[f'sum_today {pd.to_datetime(finish).date()}'])
total_plan = total_plan.join([cum_sum_yesterday, sum_today])

In [39]:
total_plan = total_plan[(total_plan[f'sum_today {pd.to_datetime(finish).date()}']>0) & (total_plan['cum_sum_yesterday'] == 0)]

In [40]:
total_plan = total_plan[[f'sum_today {pd.to_datetime(finish).date()}']]

In [41]:
worksheet_SMR.worksheet('START').clear()
gd.set_with_dataframe(worksheet_SMR.worksheet('START'), total_plan,row=1, col=1, include_index=True, include_column_header=True)

## обновление листа ПЛАН-РАБОТ-> внесение факта

In [ ]:
# list_date = list(graf_black.iloc[0, :].values)
# list_name = list(graf_black.iloc[:, 0].values)

In [ ]:
# for i in plan_PR.iloc[:, 0]:
#     index_in_plan_PR = list(plan_PR.iloc[:, 0]).index(i)
#     date_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 2]
#     date_in_plan_PR = '.'.join(reversed(str(date_in_plan_PR).split('-')))
#     value_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 1]
#     value_in_plan_PR = ','.join(value_in_plan_PR.split('.'))

#     index_row_in_graf_black = list_name.index(i) + 2
#     index_column_in_graf_black = list_date.index(date_in_plan_PR) + 1

#     worksheet_SMR.worksheet("ГРАФИК РАБОТ").update_cell(index_row_in_graf_black, index_column_in_graf_black, value_in_plan_PR)

#     print(value_in_plan_PR, date_in_plan_PR, i, index_row_in_graf_black, index_column_in_graf_black)

## обновление листа финансирование:

1. внесение плана КС, кумулятивно за отчетный период
2. акутулизация плана платежей
3. внесение сумм по фактически выполненным работам

## графики:

1. СМР план/факт
2. финансирование план/факт
3. освоение план/факт


In [ ]:
from pygsheets.chart import ChartType

In [ ]:
gz = pygsheets.authorize(service_file='/content/credentials.json')

In [ ]:
 gz = gz.open('План работ Договор Адамант')

In [ ]:
gz.worksheet_by_title('PLAN_WEEK').add_chart(('A2', 'A44'), [('B2', 'B44')], chart_type=ChartType.LINE, title='MyChart', anchor_cell='A50')

<Chart LINE 'MyChart'>

# **Обновление графика Закачика**

* позиции имеющие шифр обновляются по шифру
* позиции без шифра обновляются после фильтрации по шифру

## предобработка графика Заказчика

In [42]:
#  список из графика заказчика
list2 = df_graph['Unnamed: 13']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list3 = df_graph['Unnamed: 2']
list3 = list3.map(lambda z: str(z).lstrip())
list3 = list3.to_list()

## обновление СМР

* пайплайн по каждому контракту СМР одинаковый

### пересечение списков видов работ

In [43]:
#  список из шиифров из 'График работ'
list1 = graf.iloc[:, 0].values
list1

array(['PLAN', '***', 'VZO_POS1_POD_MOB_.', 'VZO_POS1_POD_VRD_DP17.01',
       'VZO_POS1_POD_VRS_ES0.01', 'VZO_POS1_POD_VRS_VK0.01',
       'VZO_POS1_POD_VRD_DP17.02', 'VZO_POS1_POD_VRZ_ARM.02',
       'VZO_POS1_POD_VRZ_ARM.03', 'VZO_POS1_POD_VRD_DP17.04',
       'VZO_POS1_POD_VRZ_MBT.01', 'VZO_POS1_POD_POZ_PSC.01',
       'VZO_POS1_POD_INF_STN.01', 'VZO_POS1_POD_VRZ_BOF.01',
       'VZO_POS1_POD_VRZ_BGO.02', 'VZO_POS1_POD_MMC_ONL.02',
       'VZO_POS1_POD_VRS_PES.01', 'VZO_POS1_POD_VRS_PVD.01',
       'VZO_POS1_POD_VRZ_WC.02', 'VZO_POS1_POD_CLN_BOF.01',
       'VZO_POS1_POD_CLN_BGO.01', 'VZO_POS1_POD_CLN_DER.01',
       'VZO_POS1_POD_VRS_PIT.01', 'VZO_POS1_POD_SEC_ADS.01',
       'VZO_POS1_POD_CLN_GAR.01', 'VZO_POS1_POD_CLN_SUT.01',
       'VZO_POS1_POD_CLN_SUB.01', 'PRZ_PPGR_GDS_NLS_ADS.01',
       'VZO_POS1_POD_KRA_FPK.01', '', '', '', '', '', '', '', '',
       'VZO_POS1_POD_KRA_FPK.02', 'VZO_POS1_POD_VRD_SCH.01',
       'VZO_POS1_POD_VRD_PAG.01', 'VZO_POS1_POD_VRD_PAG.02',
       

In [44]:
result = list(set(list1) & set(list2))
result

['OOS_KZ01_BK0_BFP_ARM.01',
 'VZO_POS1_POD_VRS_ES0.01',
 'OOS_AR4___.',
 'VZO_POS1_POD_VRZ_ARM.02',
 'VZO_POS1_POD_CLN_BGO.01',
 'VZO_POS1_POD_VRZ_BOF.01',
 'OOS_KZ09_BK1_BWL_OAB30.01',
 'OOS_KR1_MK0_MRS_TR.02',
 'VZO_POS1_POD_VRD_PAG.01',
 'VZO_POS1_POD_KRA_DEM.01',
 'OOS_VK___.',
 'OOS_KR1_ZEM_GRR_KT2.08',
 'OOS_KR1_ZEM_GRR_KT2.02',
 'OOS_KZ01_POD_BPD_PES.03',
 '_RNV___.',
 'OOS_AR2___.',
 'OOS_KZ09',
 'VZO_POS1_POD_VRZ_BGO.02',
 'OOS_KM1',
 'OOS_KZ11',
 'OOS_KR1_BK0_BPU_OAB30.01',
 'OOS_KZ08',
 'OOS_KZ01_POD_BPD_SCH.03',
 'OOS_KZ07',
 'OOS_KR1_ZEM_GMR_KT2.09',
 'OOS_KR1_MK0_MRS_ST.02',
 'OOS_KZ01_POD_BPD_SCH.02',
 'OOS_KR1_ZEM_GRR_KT2.07',
 'OOS_KR1_MK0_MRS_B1.01',
 'OOS_AR3',
 'OOS_OVIK',
 'OOS_KR1_MK0_MRS_TR.04',
 'VZO_POS1_POD_KRA_ANK.01',
 'OOS_KR1_MK0_MRS_B1.04',
 'OOS_KZ01_BK0_BFP_ARM.02',
 'VZO_POS1_POD_MMC_ONL.02',
 'OOS_KZ01_IZO_GFP_CP150.02',
 'BOT_GP',
 'OOS_TH',
 'OOS_KR1_ZEM_GRR_KT2.05',
 'OOS_KR1_ZEM_GMR_KT2.02',
 'OOS_KR1_ZEM_GMR_KT2.08',
 'OOS_KR1_ZEM_GMR_KT2.01',
 '

In [45]:
for i in result:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = (pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100)
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})    

{'               Армирование фундаментной плиты': 1.0}
{'            Временные электросети - установленные на начальном этапе (включают только подключение бытового городка от точки подключения и 1 временных электрощита на территории 1 кран )': 0.5}
{'         Устройство фасада ': nan}
{'            Устройство арматурного цеха': 1.0}
{'            Ежедневная уборка бытового городка (Зарплата, расходные материалы)': 1.0}
{'            Блок-контейнер - офисное помещение (амортизация). 16 штук': 1.0}
{'               Устройство вертикальных конструкций в осях 16-18_А-И': nan}
{'               Монтаж труб-распорок в осях 14-18/Г-И': 1.0}
{'               Укладка плит ПАГ -14 1 ряд': nan}
{'               Демонтаж башенного крана №1': nan}
{'            Водоснабжение и водоотведение': nan}
{'                  8 захватка': 0.0}
{'                  2 захватка': 0.8974}
{'               Устройство песчаного основания ': 1.0}
{'            Ввод объекта в эксплуатацию ( РНВ)': nan}
{'         Уст

## обновление проектирования

* первый этап - обновить по шифрам
* второй этап - обновить по наименованию работ

In [46]:
perevod = {
    'STATUS A': 1,
    'STATUS B': 0.85,
    'STATUS C': 0.6,
    'WORK': 0.3,
    'START': 0.05,
    'HOLD': 0,
    '': ''
}

In [47]:
list_PD_ID_1 = graph_PD[graph_PD.iloc[:, 1].values != ''].iloc[:,2]
list_PD_name = graph_PD[graph_PD.iloc[:, 1].values == ''].iloc[:,3]
list_PD = list_PD_ID_1.append(list_PD_name)
list_PD

77                                         608AR-20-Р-ГП
83                                         608AR-20-Р-АР
86                                        608AR-20-Р-АР1
87                                        608AR-20-Р-АР2
88                                        608AR-20-Р-АР3
                             ...                        
243    Первчиное консультирование по СМР проведение т...
244                         Сопровождение строительства 
245                Отчет по вводу объекта в эксплуатацию
246                        Энергетическое моделирование 
247                                      Итоговый аудит 
Length: 248, dtype: object

In [48]:
result_PD_ID = list(set(list_PD) & set(list2))
result_PD_name = list(set(list_PD) & set(list3))

In [49]:
for i in result_PD_ID:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]})

{'            Вертикальные несущие конструкции второго этажа 608AR-20-Р-КЖ-5-1': 1}
{'            Вертикальные конструкции четвёртого этажа 608AR-20-Р-КЖ-9-2': 0}
{'            Гидроизоляция подземных конструкций': 1}
{'            Наружное теплоснабжение': 0.85}
{'            Плита перекрытия над четвёртым этажом 608AR-20-Р-КЖ-10-3': 0.6}
{'            Автоматическая установка газового пожаротушения': 0}
{'            Внутренние системы канализации': 0.6}
{'            Наружные сети связи': 0}
{'            Подземная часть. Вертикальные конструкции. 608AR-20-Р-КЖ-1-3': 0.85}
{'            Вертикальные конструкции пятого этажа 608AR-20-Р-КЖ-11-1': 0}
{'            Технологические решения. Предприятия общественного питания': 0.05}
{'            Технологические решения актового зала': 0.05}
{'            Вентиляция общеобменная': 0.3}
{'            Система оповещения и управления эвакуацией': 0.3}
{'            Однолинейные схемы распределительных щитов': 0}
{'            Автоматизирован

In [50]:
for i in result_PD_name:
    df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]
    print({df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]}) 

{'            Общие требования к ИТ-инфраструктуре': 0.3}
{'            Система позиционирования': 1}
{'            Эскизный проект АИ: План и спецификация сантехники и аксессуаров': 0.6}
{'            Водоснабжение и водоотведение': ''}
{'         Инженерное оборудование и системы': ''}
{'               Слаботочные системы': 0.85}
{'         Технология': ''}
{'               Предоставление сводной ВОР по объекту': 0}
{'            Утверждение растановки светильников наружных': 0}
{'            План благоустройства, МАФЫ, озеленение, ограждение': 0.3}
{'            Пожарная сигнализация': 0.3}
{'            Задание для раздела СС': 0.85}
{'            Технологические решения школы.': 0}
{'            Интерьеры. Полы': 0.3}
{'               Конструктивные решения': ''}
{'            Эскизный проект АИ: План и спецификация полов': 0.6}
{'            Задание для раздела ЭОМ': 0.85}
{'         Дизайн-проект интерьеров': 1}
{'            Фасады,архитектурная подсветка': 0}
{'            Раз

создание файла с обновленными данными

In [51]:
df_graph.to_excel('Сводный план-график {}.xlsx'.format('20.02.2023'), sheet_name='Обновление от {}'.format('20.02.2023'),index=False)